# 2D Backends

In this tutorial we are going to examine the 2D capabilities of this plotting module, whose major objective is to integrate modern interactive plotting libraries with SymPy. We are also going to illustrate a couple of important capabilities:
1. Detection and plotting of singularities.
2. Visualization of `Piecewise` expressions.

## Comparison between 2D backends

Let's start by comparing the 2D backends. First, we will show the results with Matplotlib, and explain the limitations. Then we will show what a different backend can do.

In [ ]:
%matplotlib widget
from sympy import *
from spb import *
var("u, v, x, y")

In the above code cell we first imported almost all plotting function. Specifically, we imported:

* `plot`
* `plot_polar`
* `plot_list`
* `plot_geometry`
* `plot_piecewise`
* `plot_parametric`
* `plot3d`
* `plot3d_parametric_line`
* `plot3d_parametric_surface`
* `plot3d_implicit`
* `plot_contour`
* `plot_implicit`
* `plot_vector`
* `plot_complex`
* `plot_real_imag`
* `plot_complex_list`
* `plot_complex_vector`

The user can explore the documentation associated to each function by executing `help(FUNCTION_NAME)`.

There are also two more functions that were not imported, as they depend on `panel` which is a slow module to load: `plotgrid` and `iplot`. We will talk more about the latter in [tutorial 5](tutorial-5.simple-parametric-interactive-plots.ipynb).

**Remember**: while many of the above functions are identical to the ones from `sympy`, they are not compatible when using a different backend!

We also imported the aliases of all backends. The following backends are available from the following submodules:

|       Sub-module      | Backend             | Alias |
|:---------------------:|:-------------------:|:-----:|
| backends.bokeh      | BokehBackend      |  BB |
| backends.matplotlib | MatplotlibBackend |  MB |
| backends.plotly     | PlotlyBackend     |  PB |
| backends.k3d        | K3DBackend        |  KB |

Only `MatplotlibBackend`, `BokehBackend` and `PlotlyBackend` support 2D plots.

In [ ]:
p = plot(sin(x), cos(x), log(x), backend=MB)

In the previous command we specified the optional keyword argument `backend=`. If not provided, the default backend will be used. Refer to [tutorial 3](tutorial-3.customize-the-module.ipynb) to learn how to customize the module and set a different default backend.

We can see a `RuntimeWarning` in the output cell: it was generated by the evaluation algorithm while processing $\log{(x)}$, which is only defined for $x > 0$, whereas we asked the plot function to evaluate it over the interval $-10 \le x \le 10$.

Once we plot multiple expression simultaneously, the legend will automatically show up. We can disable it by setting `legend=False`. Note the italic texts in the labels and legend: `MatplotlibBackend` will render latex by default. We can turn off this behaviour by setting `use_latex=False`, in which case the string representation will be used instead.

Note that:
* In order to interact with the plot we have to use the buttons on the toolbar.
* If we move the cursor over the figure, we can see its coordinates. By moving it over a line we only get approximate coordinates.

With the previous command, we plotted 3 different expressions. Therefore, the plot object `p` contains 3 data series. We can easily access the data series by using the index notation: this is useful in order to extract numerical data as we will see in [tutorial 4](tutorial-4.creating-custom-plots.ipynb).

In [ ]:
print(p)
print("\nInformation about the first series:")
print(p[0])

Let's visualize the same plot with `PlotlyBackend`:

In [ ]:
plot(sin(x), cos(x), log(x), backend=PB)

The top toolbar can be used to interact with the plot. However, there are more natural ways:
* Click and drag to zoom into a rectangular selection.
* Move the cursor in the middle of the horizontal axis, click and drag to pan horizontally.
* Move the cursor in the middle of the vertical axis, click and drag to pan vertically.
* Move the cursor near the ends of the horizontal/vertical axis: click and drag to resize.
* Move the cursor over a line: a tooltip will show the coordinate of that point in the data series. Note that there is no interpolation between two consecutive points.
* Click over a label in the legend to hide/show that data series.

Let's now visualize the same plot with BokehBackend:

In [ ]:
plot(sin(x), cos(x), log(x), backend=BB)

Here, we can:
* Click and drag to pan the plot around. **Once we are done panning, the plot automatically updates all the data series according to the new range**. This is a wonderful feature of Bokeh, which allows us to type less and explore more.
* Click and drag the axis to pan the plot only on one direction.
* Click the legend entries to hide/show the data series.
* Move the cursor over a line: a tooltip will show the coordinate of that point in the data series.
* Use the toolbar to change the tool, for example we can select the _Box Zoom_ to zoom into a rectangular region.

Is some occasion, it might be helpful to assign a custom label to a specific expression. We can do that in the following way:

In [ ]:
p = plot((cos(x), "$f_{1}$"), (sin(x), "$f_{2}$"), (log(x), "$f_{3}$"),
         backend=MB, legend=True, title="Latex Support",
         xlabel=r"$\xi$", ylabel=r"$\eta$", detect_poles=False)

At the time of writing this, there might be problems when using [Plotly with Latex on Firefox](https://github.com/plotly/plotly.js/issues/5374).

Also, note that, differently from Matplotlib and Plotly, Bokeh doesn't support Latex!

## Adaptive Algorithm and Singularity Detection

Contrary to the SymPy plotting module, this module allows us to correctly plot singularities (or at least try to). By default, this detection is turned off, for example:

In [ ]:
plot(tan(x), (x, -10, 10), backend=PB)

As we can see, the plot is hardly readable:
* There is a pretty huge data range covered by the y-axis, which is "flattening" the non-linearities.
* By zooming in, we can see a continous line that connects the points even through singularities.

We can turn-on the singularity detection by setting `detect_poles=True`, which is going to post-process the numerical data (no symbolic analysis is done).

In [ ]:
plot(tan(x), (x, -10, 10), backend=PB, detect_poles=True)

Here:
* There is a clear separation where the singularities are located.
* There is a bias in the y-axis: data is not centered. To correct it, we can set `ylim`.

2D plot functions implements two way to generate numerical data:

* uniform sampling: the x-range is uniformly discretized with `n` points.
* adaptive sampling: [an algorithm](https://github.com/python-adaptive/adaptive/) is going to choose where to sample the function. This is the default behaviour.

It is important to realize that the finer the discretization of the domain, the better the detection. Therefore, it might be necessary to set a lower goal in the adaptive algorithm (`adaptive_goal=0.01` is the default value), or set `adaptive=False` and `n=2000` (some arbitrary large number). Also, as a last resort, one might also change the value of `eps` (default to 0.1).

Note that there is a bias in the y-axis: it is not perfectly centered. We can easily fix it by scrolling the y-axis or by setting the `ylim` keyword:

In [ ]:
plot(tan(x), (x, -10, 10), backend=PB, adaptive_goal=0.001, detect_poles=True, ylim=(-7, 7))

When ``adaptive=True`` (default value), the plot functions are deterministic in the sense that the same numerical data are generated, as long as ``adaptive_goal`` remain the same.

The default value of ``adaptive_goal=0.01`` represents a good trade-off between smoothness and performance. If we believe that a mathematical expression is not sufficiently resolved in the figure, we can either:

1. decrease the value of ``adaptive_goal`` at the cost of performance.
2. set ``adaptive=False`` and increase the number of discretization points ``n``.

For illustrative purposes, let's examine the following example:

* The first plot uses the adaptive algorithm, but its goal is not enough to properly resolve the mathematical expression.
* The second plot also uses the adaptive algorithm, but with an appropriate goal.
* The third plot uses the uniform sampling.

In [ ]:
alpha = S(1) / 10
expr = exp(-abs(x)**alpha) + exp(-abs(1 - x)**alpha)

p1 = plot(expr, (x, -0.1, 1.1), adaptive=True, adaptive_goal=0.5)
p2 = plot(expr, (x, -0.1, 1.1), adaptive=True, adaptive_goal=0.01)
p3 = plot(expr, (x, -0.1, 1.1), adaptive=False, n=1e04)

Note the differences between the second and third output. The maximum y-values computed in the second plot are greater than the third. That's because the adaptive algorithm got closer to the singularity thanks to the chosen ``adaptive_goal``.

## Plotting Piecewise functions

We can also plot `Piecewise` expressions, eventually showing the discontinuities. For example:

In [ ]:
pf = Piecewise(
    (sin(x), x < -5),
    (2, Eq(x, 0)),
    (3, Eq(x, 2)),
    (cos(x), (x > 0) & (x < 2)),
    (x / 2, True)
)
display(pf)
plot_piecewise(pf, backend=MB)

As a design choice, the algorithm is going to extract the different pieces and plot them separately. Note that the end of each range are visible too!

When using ``BokehBackend`` to plot piecewise functions, the user must set `update_event=False` in order to turn-off the automatic update when panning. Failing to do so will result in recomputing the series and override the original ranges, thus leading to an incorrect plot:

In [ ]:
plot_piecewise(pf, backend=BB, update_event=False)

Now, let's consider a case where plotting a ``Piecewise`` expression might fail:

In [ ]:
f = real_root((log(x / (x - 2))), 3)
display(f)
plot_piecewise(f)

In such cases, we can try to further manipulate the `Piecewise` expression in order to substitute the problematic condition with equivalent conditions. Or we can use the `plot` function, keeping in mind that it doesn't correctly display discontinuities:

In [ ]:
plot(f)

## Combining Plots

Let's consider two different plots:

In [ ]:
p1 = plot(sin(x), cos(x), backend=MB)
p2 = plot(log(x), exp(-x / 5) * sin(5 * x), (x, 1e-05, 10),
          backend=MB, rendering_kw=dict(linestyle="--"))

We used the `rendering_kw` dictionary to provide custom options that have been passed to Matplotlib. Type `help(MB)` (or any other backend) to discover more customization options. With this dictionary, we can also provide a specific color!

In case we need to access these data series we can use the index notation:

In [ ]:
s1 = p1[0]
type(s1)

Data series are responsible to generate the numerical data, which can be extracted with the `get_data` method:

In [ ]:
data = s1.get_data()

We can combine 2 plot objects in 3 ways:

1. by calling `p1.extend(p2)`: this method copy all the data series from `p2` into `p1`.
2. by calling the `p1.append(p2[idx])`: copy the data series ad index `idx` of `p2` and append it to `p1`.
3. by summing plot objects `p1 + p2`: create a new plot copying all the data series from `p1` and `p2` and also merge the keyword arguments of the two plots.

Note that with 1. and 2., the figure gets updated only after calling the `show` method.

Let's try the first way. Note that the keyword arguments of `p2` are lost:

In [ ]:
p1.extend(p2)
p1.show()

Note that the two series of `p2` maintained the specified line style, but they received a new line color to make the plot more readable.

Let's append a data series from `p2` into `p1`:

In [ ]:
p1 = plot(sin(x), cos(x), backend=MB, show=False)
p2 = plot(log(x), exp(-x / 5) * sin(5 * x), (x, 1e-05, 10),
          backend=MB, rendering_kw=dict(linestyle="--"), show=False)
p1.append(p2[1])
p1.show()

Finally, let's add the plot objects:

In [ ]:
p1 = plot(sin(x), cos(x), backend=MB, show=False)
p2 = plot(log(x), exp(-x / 5) * sin(5 * x), (x, 1e-05, 10),
          backend=MB, rendering_kw=dict(linestyle="--"), show=False)
p3 = (p1 + p2)
p3.show()

## Saving Plots

Generally, there are two ways to save a plot:

1. Manually, by clicking the save button in the toolbar. For Matplotlib, this only works if the magic line `%matplotlib widget` has been executed.
2. Programmatically, by calling the `save` method of a plot object. This method is just a wrapper to the `save` method exposed by the actual plotting library, therefore we can save jpg, png, pdf, svg or html files if the library supports these functionality.

Note that some backends requires additional dependencies to be installed in order to export pictures. Run the following command and follow the links in the Reference section to learn more.

In [ ]:
help(MB.save)

In [ ]:
help(PB.save)

In [ ]:
help(BB.save)

## Setting custom color loops

We can also change the color loop used by the backend. All we have to do is to set a list of colors to the `colorloop` class attribute. Let's start by creating a plot and visualizing the default color loop:

In [ ]:
_plot = lambda B: plot(sin(x) / 3, 2 * sin(x) / 3, sin(x), backend=B, adaptive=False, n=100)

import matplotlib.cm as cm
_plot(MB)

Now, let's change to another matplotlib's colormap. Note that `colorloop` must use a list of colors:

In [ ]:
MB.colorloop = cm.Set1.colors
_plot(MB)

We can also use colorloops from a different plotting library:

In [ ]:
import plotly.express as px
MB.colorloop = px.colors.qualitative.Plotly
_plot(MB)

Finally, we can manually write a list of colors:

In [ ]:
MB.colorloop = ["red", "orange", "gold"]
_plot(MB)

Note that colorloops designed for a different backend might not work. For example, it is impossible to set a matplotlib-specific color loop to Plotly:

In [ ]:
PB.colorloop = cm.tab10.colors
_plot(PB) 